# 0. Imports

In [1]:
# # ideias adicionais:
#     - criar coluna gênero de quem anunciou
#     - criar coluna CEP
#     -

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 1000)

In [2]:
!pip install fancyimpute

# 1. Coletar dados

In [3]:
# <explicar como foi feita a coleta de dados e onde localizar .py>

| Campop               | Descrição                                      |
|----------------------|--------------------------------------------------|
| find_anuncio         | o titulo do anuncio                              |
| find_condominio      | o preço do condominio                            |
| find_descri          | a descrição feita do imovel                      |
| find_endereco        | Endereço = bairro, Cidade, estado, cep           |
| find_iptu            | IPTU                                             |
| find_loc             | geralmente a rua                                 |
| find_m_quadrado      | Tamanho do imóvel em metros quadrados            |
| find_price           | Preço do imóvel                                  |
| find_vendedor        | Informações sobre o vendedor                     |
| is_academia          | Indica se há academia/centro de fitness          |
| is_apartamento       | Indica se é um apartamento                       |
| is_ar_condicionado   | Indica se tem ar condicionado                     |
| is_churrasqueira     | Indica se há churrasqueira                       |
| is_elevador          | Indica se há elevador                            |
| is_mobiliado         | Indica se o imóvel está mobiliado                |
| is_piscina           | Indica se há piscina                             |
| is_portaria          | Indica se há portaria/serviço de portaria        |
| is_profissional      | Indica se o vendedor é um profissional           |
| is_salao_festa       | Indica se há salão de festas/área de eventos     |
| is_varanda           | Indica se há varanda                             |
| n_banheiros          | Número de banheiros                              |
| n_fotos              | Número de fotos no anúncio                       |
| n_garagens           | Número de vagas/garagens                         |
| n_quartos            | Número de quartos                                |

In [7]:
path_raw = '/content/scraped_data.csv'
df_raw = pd.read_csv(path_raw, sep=',')
df_raw = df_raw.sample(5000)

In [8]:
df_raw.head(3)

,find_anuncio,find_condominio,find_descri,find_endereco,find_iptu,find_loc,find_m_quadrado,find_price,find_vendedor,is_academia,is_apertamento,is_ar_condicionado,is_churrasqueira,is_elevador,is_mobiliado,is_piscina,is_portaria,is_profissional,is_salao_festa,is_varanda,n_banheiros,n_fotos,n_garagens,n_quartos
26445,"Apartamento em Boa Viagem | stúdio, 1 ou 2 qua...",R$ 1 / mês,"Para mais informações, chama no Whatsapp: 81-....","Boa Viagem, Recife, PE, 51030020",R$ 1,Rua Visconde de Jequitinhonha - até 2542 - lad...,26m²,265000,Ribeiro Imóveis,True,True,False,False,True,False,True,True,True,True,False,1,20,1,1
56933,"Apartamento à venda, 180 m² por R$ 740.000,00 ...",R$ 0 / mês,Código do anúncio: AP11668\n\nEDF. MARIA CLARA...,"Espinheiro, Recife, PE, 52020000",R$ 0,Avenida Governador Agamenon Magalhães - de 2 a...,180m²,740000,Bellolar Imoveis,False,True,False,False,True,False,False,False,True,False,False,1,16,2,4
68965,vrApartamento para venda possui 237 metros qua...,R$ 0 / mês,Código do anúncio: vr-065\n\n.8.1.9.8.8.3.4.5....,"Graças, Recife, PE, 52011160",R$ 0,Rua Dom Sebastião Leme,237m²,1499000,Dream Residence Imóveis Recife,True,True,True,True,True,False,True,True,True,True,True,5 ou mais,20,NaN,4


# 2. Realizar pré-processamento dos dados

In [9]:
df_raw.shape

(5000, 24)

In [10]:
df_0 = df_raw.copy()

In [11]:
# vamos considerar como chave todas as colunas menos
df_0 = df_0.drop_duplicates(subset=['find_condominio','find_endereco','find_iptu','find_loc', 'find_m_quadrado', 'find_price', 'find_vendedor', 'is_academia', 'is_apertamento', 'is_ar_condicionado', 'is_churrasqueira', 'is_elevador', 'is_mobiliado', 'is_piscina', 'is_portaria', 'is_profissional', 'is_salao_festa', 'is_varanda', 'n_banheiros', 'n_fotos', 'n_garagens', 'n_quartos'])

In [12]:
set(df_0.columns) - set(['find_condominio','find_endereco','find_iptu','find_loc', 'find_m_quadrado', 'find_price', 'find_vendedor', 'is_academia', 'is_apertamento', 'is_ar_condicionado', 'is_churrasqueira', 'is_elevador', 'is_mobiliado', 'is_piscina', 'is_portaria', 'is_profissional', 'is_salao_festa', 'is_varanda', 'n_banheiros', 'n_fotos', 'n_garagens', 'n_quartos'])

# chave primária vai ser todas as colunas menos o que texto que tem no anuncio e na descrição dele

{'find_anuncio', 'find_descri'}

In [13]:
df_0.shape

(4858, 24)

## 2.0 <dar nome melhor para descrever essas alterações iniciais>

### 2.0.1 Algumas classes são muito uteis como valor numerico, mas não podem ser totalmente convertidas. exemplo: o numero de quartos tem o campo '5 ou mais' vamos fazer o drop desses valores

In [14]:
df_0 = df_0[df_0['n_quartos'] != '5 ou mais']
df_0 = df_0[df_0['n_banheiros'] != '5 ou mais']
df_0 = df_0[df_0['n_garagens'] != '5 ou mais']

## 2.1 definição de tipos e criação de novas colunas derivadas

In [15]:
df_0.dtypes

find_anuncio          object
find_condominio       object
find_descri           object
find_endereco         object
find_iptu             object
find_loc              object
find_m_quadrado       object
find_price             int64
find_vendedor         object
is_academia             bool
is_apertamento          bool
is_ar_condicionado      bool
is_churrasqueira        bool
is_elevador             bool
is_mobiliado            bool
is_piscina              bool
is_portaria             bool
is_profissional         bool
is_salao_festa          bool
is_varanda              bool
n_banheiros           object
n_fotos                int64
n_garagens            object
n_quartos             object
dtype: object

In [16]:
# deixando como num a coluna find_condominio
df_0['find_condominio'] = df_0['find_condominio'].astype('str').str.replace('R$', '').str.replace(' / mês', '').str.replace('.', '')

# deixando como num a coluna find_iptu
df_0['find_iptu'] = df_0['find_iptu'].astype('str').str.replace('R$ ', '').str.replace('.', '')

# deixando como num a coluna find_m_quadrado
df_0['find_m_quadrado'] = df_0['find_m_quadrado'].astype('str').str.replace('m²', '').str.replace('.', '')

# cast direto colunas(floats devido a conseguir dar cast em NaN)
df_0['find_anuncio'] = df_0['find_anuncio'].astype('category')
df_0['find_condominio'] = df_0['find_condominio'].astype('float')
df_0['find_descri'] = df_0['find_descri'].astype('category')
df_0['find_endereco'] = df_0['find_endereco'].astype('category')
df_0['find_iptu'] = df_0['find_iptu'].astype('float')
df_0['find_loc'] = df_0['find_loc'].astype('category')
df_0['find_m_quadrado'] = df_0['find_m_quadrado'].astype('float')
df_0['find_price'] = df_0['find_price'].astype('float')
df_0['find_vendedor'] = df_0['find_vendedor'].astype('category')
df_0['is_academia'] = df_0['is_academia'].astype('category')
df_0['is_apertamento'] = df_0['is_apertamento'].astype('category')
df_0['is_ar_condicionado'] = df_0['is_ar_condicionado'].astype('category')
df_0['is_churrasqueira'] = df_0['is_churrasqueira'].astype('category')
df_0['is_elevador'] = df_0['is_elevador'].astype('category')
df_0['is_mobiliado'] = df_0['is_mobiliado'].astype('category')
df_0['is_piscina'] = df_0['is_piscina'].astype('category')
df_0['is_portaria'] = df_0['is_portaria'].astype('category')
df_0['is_profissional'] = df_0['is_profissional'].astype('category')
df_0['is_salao_festa'] = df_0['is_salao_festa'].astype('category')
df_0['is_varanda'] = df_0['is_varanda'].astype('category')
df_0['n_banheiros'] = df_0['n_banheiros'].astype('float')
df_0['n_fotos'] = df_0['n_fotos'].astype('float')
df_0['n_garagens'] = df_0['n_garagens'].astype('float')
df_0['n_quartos'] = df_0['n_quartos'].astype('float')


In [17]:
df_0.dtypes

find_anuncio          category
find_condominio        float64
find_descri           category
find_endereco         category
find_iptu              float64
find_loc              category
find_m_quadrado        float64
find_price             float64
find_vendedor         category
is_academia           category
is_apertamento        category
is_ar_condicionado    category
is_churrasqueira      category
is_elevador           category
is_mobiliado          category
is_piscina            category
is_portaria           category
is_profissional       category
is_salao_festa        category
is_varanda            category
n_banheiros            float64
n_fotos                float64
n_garagens             float64
n_quartos              float64
dtype: object

In [18]:
df_0[df_0.select_dtypes(['category']).columns] = df_0.select_dtypes(['category']).apply(lambda x: x.cat.codes)

In [19]:
df_0.dtypes

find_anuncio            int16
find_condominio       float64
find_descri             int16
find_endereco           int16
find_iptu             float64
find_loc                int16
find_m_quadrado       float64
find_price            float64
find_vendedor           int16
is_academia              int8
is_apertamento           int8
is_ar_condicionado       int8
is_churrasqueira         int8
is_elevador              int8
is_mobiliado             int8
is_piscina               int8
is_portaria              int8
is_profissional          int8
is_salao_festa           int8
is_varanda               int8
n_banheiros           float64
n_fotos               float64
n_garagens            float64
n_quartos             float64
dtype: object

## 2.2 Tratamento de dados ausentes
  - vamos fazer a imputação das únicas 3 colunas que apresentam preenchimento vazio
    - find_m_quadrado
    - n_banheiros
    - n_garagens

In [27]:
df_0.isna().sum()

find_anuncio            0
find_condominio         0
find_descri             0
find_endereco           0
find_iptu               0
find_loc                0
find_m_quadrado        20
find_price              0
find_vendedor           0
is_academia             0
is_apertamento          0
is_ar_condicionado      0
is_churrasqueira        0
is_elevador             0
is_mobiliado            0
is_piscina              0
is_portaria             0
is_profissional         0
is_salao_festa          0
is_varanda              0
n_banheiros             9
n_fotos                 0
n_garagens            109
n_quartos               0
dtype: int64

### 2.2.1 Removendo colunas pelo knnimputer

In [28]:
from fancyimpute import KNN

df_0_knn = KNN(k=2).fit_transform(df_0)

Imputing row 1/4538 with 0 missing, elapsed time: 7.913
Imputing row 101/4538 with 0 missing, elapsed time: 7.914
Imputing row 201/4538 with 0 missing, elapsed time: 7.916
Imputing row 301/4538 with 1 missing, elapsed time: 7.917
Imputing row 401/4538 with 0 missing, elapsed time: 7.918
Imputing row 501/4538 with 0 missing, elapsed time: 7.918
Imputing row 601/4538 with 0 missing, elapsed time: 7.919
Imputing row 701/4538 with 0 missing, elapsed time: 7.920
Imputing row 801/4538 with 0 missing, elapsed time: 7.921
Imputing row 901/4538 with 0 missing, elapsed time: 7.921
Imputing row 1001/4538 with 0 missing, elapsed time: 7.922
Imputing row 1101/4538 with 0 missing, elapsed time: 7.923
Imputing row 1201/4538 with 0 missing, elapsed time: 7.923
Imputing row 1301/4538 with 0 missing, elapsed time: 7.924
Imputing row 1401/4538 with 0 missing, elapsed time: 7.927
Imputing row 1501/4538 with 0 missing, elapsed time: 7.928
Imputing row 1601/4538 with 0 missing, elapsed time: 7.929
Imputing 

In [30]:
df_0 = pd.DataFrame(data=df_0_knn[0:,0:],columns=df_0.columns)


In [31]:
df_0.isna().sum()

find_anuncio          0
find_condominio       0
find_descri           0
find_endereco         0
find_iptu             0
find_loc              0
find_m_quadrado       0
find_price            0
find_vendedor         0
is_academia           0
is_apertamento        0
is_ar_condicionado    0
is_churrasqueira      0
is_elevador           0
is_mobiliado          0
is_piscina            0
is_portaria           0
is_profissional       0
is_salao_festa        0
is_varanda            0
n_banheiros           0
n_fotos               0
n_garagens            0
n_quartos             0
dtype: int64

## 2.3 Limpeza dos dados

### 2.3.1 univariado

### 2.3.2 bivariado

### 2.3.3 multivariado

# 3. Apresentar estatísticas descritivas dos dados com visualizações

# 4. Realizar teste de hipóteses para comparação de valores de categorias e visualizar diferenças